Answer questions:

1. What are the most frequent words based on weights for each article - we did already
2. Words clustering
3. Sentiment analysis.  Such as most popular....
4. Ranking
5. Key word search.  What are the most popular boots for winter 2019 - 2020?

In [1]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import numpy as np
from nltk.tokenize import RegexpTokenizer
import nltk
import spacy

In [2]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'take', 'tell', 'shake', 'turn', 'talk', 'say']
Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun ORG
Recode PRODUCT
earlier this week DATE


In [3]:
urls = ['https://www.elle.com/fashion/shopping/a29657264/winter-bag-trends-2019-2020/',
      'https://www.elle.com/fashion/personal-style/a29462082/fresh-outfit-ideas-to-wear-to-work/',
      'https://www.elle.com/fashion/shopping/a29711557/winter-shoe-trends-2019-2020/',
       'http://www.nfl.com/news/story/0ap3000001074509/article/patrick-mahomes-knee-to-start-sunday-vs-titans']

In [4]:

#Remove unwanted html element and put words in lower case
def urltotext(url):

    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)


    # set([t.parent.name for t in text])

    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',   
        # there may be more elements you don't want.
    ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
#             print("\t",t.parent.name)
#             print('{} '.format(t))

    return output.lower()


In [5]:
# Initialize new list

def prunePOS(part = None):
    
    if part is None:
        # Create tokenizer
        tokenizer = RegexpTokenizer('\w+')
        text = [tokenizer.tokenize(urltotext(url)) for url in urls]

    else:  
        nlp = spacy.load("en_core_web_sm")
        text = []
        for url in urls:
            doc = nlp(urltotext(url))
            text.append([token.text for token in doc if token.tag_ == part])
        
    # Add to words_ns all words that are in words but not in sw
    corpus = []
    sw = nltk.corpus.stopwords.words('english')
    for item in text:
        corpus.append(" ".join([word for word in item if word not in sw]))
     
    return corpus

In [6]:
corpus = prunePOS()

vectorizer = TfidfVectorizer(lowercase = False)
wordweights = vectorizer.fit_transform(corpus)

corpus = prunePOS("JJ")

vectorizer_JJ = TfidfVectorizer(lowercase = False)
wordweights_JJ = vectorizer_JJ.fit_transform(corpus)


# corpus = prunePOS("NP")

# vectorizer_NP = TfidfVectorizer(lowercase = False)
# wordweights_NP = vectorizer_NP.fit_transform(corpus)


In [7]:
#toDense() might potentailly cause memory issue
searchindex = vectorizer.get_feature_names().index('bag')
searchweight = wordweights[:,searchindex].todense()

reweightedwords = np.asarray(searchweight).T @ np.asarray(wordweights_JJ.todense())


list(reversed(sorted((w,word) for w,word in zip(reweightedwords.reshape(-1), vectorizer_JJ.get_feature_names()))))[:30]


[(0.1715621420947476, 'top'),
 (0.11913242393272831, 'pouch'),
 (0.08934931794954624, 'soft'),
 (0.08934931794954624, 'small'),
 (0.08934931794954624, 'delicate'),
 (0.08339096441397625, 'favorite'),
 (0.07332112540782758, 'new'),
 (0.07188257697837777, 'structured'),
 (0.07044402854892796, 'next'),
 (0.06813726924466286, 'digital'),
 (0.059566211966364156, 'smooth'),
 (0.059566211966364156, 'modern'),
 (0.05009715544147855, 'oversized'),
 (0.048401234128735114, 'cute'),
 (0.04696268569928531, 'general'),
 (0.04542484616310857, 'main'),
 (0.044260217229725285, 'chic'),
 (0.029783105983182078, 'worthy'),
 (0.029783105983182078, 'supple'),
 (0.029783105983182078, 'spacious'),
 (0.029783105983182078, 'slouchy'),
 (0.029783105983182078, 'premillennial'),
 (0.029783105983182078, 'popular'),
 (0.029783105983182078, 'photographed'),
 (0.029783105983182078, 'multiple'),
 (0.029783105983182078, 'minimalistic'),
 (0.029783105983182078, 'mikado'),
 (0.029783105983182078, 'micro'),
 (0.02978310598

In [14]:
# Import RegexpTokenizer from nltk.tokenize
from nltk.tokenize import RegexpTokenizer

# Create tokenizer
tokenizer = RegexpTokenizer('\w+')

# Create tokens
tokens = tokenizer.tokenize(output)
tokens[:30]

NameError: name 'output' is not defined

In [15]:
Counter(tokens)
list(reversed(sorted([(count, word) for word, count in Counter(tokens).items()])))[:20]

NameError: name 'tokens' is not defined

In [16]:
# Initialize new list
words = []


# Loop through list tokens and make lower case
for word in tokens:
    words.append(word.lower())


# Print several items from list as sanity check
words[:30]

NameError: name 'tokens' is not defined

In [17]:
import nltk

In [18]:
# Get English stopwords and print some of them
sw = nltk.corpus.stopwords.words('english')
sw[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [19]:
# Initialize new list
words_ns = []

# Add to words_ns all words that are in words but not in sw
for word in words:
    if word not in sw:
        words_ns.append(word)

# Print several list items as sanity check
words_ns[:30]

[]